In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder


from itertools import product
import gc
import random as python_random
import pickle


In [2]:
# Keras seeding to produce reproduciable results
np.random.seed(123)
python_random.seed(123)
tf.random.set_seed(1234)


In [3]:
# Utility Functions

def downcast_dtypes(df):
    '''
        Changes column types in the dataframe: 
                
                `float64` type to `float32`
                `int64` and `int32` to `int16`
    '''
    
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype in ["int32", "int64"]]
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float16)
    df[int_cols]   = df[int_cols].astype(np.int16)
    
    return df


def lag_feature(df, lags, col):
    tmp = df[['date_block_num','shop_id','item_id',col]]
    for i in tqdm(lags):
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','shop_id','item_id', col+'_lag_'+str(i)]
        shifted['date_block_num'] += i
        df = pd.merge(df, shifted, on=['date_block_num','shop_id','item_id'], how='left').fillna(0)
    return df

### Load Data

In [4]:
# Read the data into variable

train = pd.read_csv('./final_project_data/sales_train.csv')
items = pd.read_csv('./final_project_data/items.csv')
category = pd.read_csv('./final_project_data/item_categories.csv')
shops = pd.read_csv('./final_project_data/shops.csv')
test = pd.read_csv('./final_project_data/test.csv')

In [5]:
train = downcast_dtypes(train)
sales = train.copy()
#sales_train = train.copy() 
sales_test = test.copy()


In [6]:
# # Remove outliers

# sales_train = sales_train[(sales_train.item_price<100000) & (sales_train.item_cnt_day<1001)]
# sales_train.loc[sales_train.item_price<0, 'item_price'] = 0

# sales_train.head(2).T

In [7]:
items.iloc[0]

item_name           ! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D
item_id                                                     0
item_category_id                                           40
Name: 0, dtype: object

In [8]:
category.iloc[0]

item_category_name    PC - Гарнитуры/Наушники
item_category_id                            0
Name: 0, dtype: object

In [9]:
shops.iloc[0]

shop_name    !Якутск Орджоникидзе, 56 фран
shop_id                                  0
Name: 0, dtype: object

In [10]:
sales_test.head(2).T

,0,1
ID,0,1
shop_id,5,5
item_id,5037,5320


In [11]:
print(f' Items in train:{sales.item_id.nunique()}\t Items in test: {sales_test.item_id.nunique()}')
print(f' Shops in train:{sales.shop_id.nunique()}\t Shops in test: {sales_test.shop_id.nunique()}')

# Shops in test not in train
print(len(np.setdiff1d(sales_test.shop_id.unique(),sales.shop_id.unique())))

# Items in test not in train
print(len(np.setdiff1d(sales_test.item_id.unique(),sales.item_id.unique())))

 Items in train:21807	 Items in test: 5100
 Shops in train:60	 Shops in test: 42
0
363


In [12]:
# # Sale figures for each item

# sales_item_id = (sales_train.groupby(['date_block_num', 'item_id'],as_index=False)['item_cnt_day']
#                   .sum().rename(columns = {'item_cnt_day':'item_cnt_month'}))

# # Items with zero sales in last 6 months
# sold_items = sales_item_id.loc[(sales_item_id.date_block_num > 26
#                                ) & (sales_item_id.item_cnt_month > 0)].item_id.unique()
# no_sales = sales_train.item_id.nunique() - len(sold_items)

# print(f'Items with no sales in last 6 months in train: {no_sales}')
# print(f'Items with no sales in last 6 months in test: ' +
#        f'{len(np.setdiff1d(sales_test.item_id.unique(),sold_items))}')


In [13]:
# # Sale figures for each shop
# sales_shop_id = (sales_train.groupby(['date_block_num', 'shop_id'],as_index=False)['item_cnt_day']
#                   .sum().rename(columns = {'item_cnt_day':'item_cnt_month'}))

# # Items with zero sales in last 6 months
# sold_shops = sales_shop_id.loc[(sales_shop_id.date_block_num > 26
#                                ) & (sales_shop_id.item_cnt_month > 0)].shop_id.unique()
# no_sales = sales_train.shop_id.nunique() - len(sold_shops)

# print(f'Shops with no sales in last 6 months in train: {no_sales}')
# print(f'Shops with no sales in last 6 months in test: ' +
#        f'{len(np.setdiff1d(sales_test.shop_id.unique(),sold_shops))}')


In [14]:
# # Prepare test data for appending to train

# # Store ID column in another df as it will be dropped before appending
# sales_index = sales_test['ID']

# sales = (sales_train.groupby(['date_block_num','shop_id','item_id'])
#                  .agg({'item_cnt_day': 'sum', 'item_price':'mean' })
#                  .rename(columns = {'item_cnt_day':'item_cnt_month'})
#                  .reset_index())

In [15]:
sales['revenue'] = sales['item_price'] * sales['item_cnt_day']

In [16]:
# item_sales_last_month = sales.loc[sales.date_block_num == 33].drop(['date_block_num'], axis=1)

# sales_test['date_block_num'] = 34
# sales_test = sales_test.merge(item_sales_last_month, on=['shop_id', 'item_id'], how='left').fillna(0)
# sales_test.drop('ID', axis=1, inplace=True)

# sales = sales.append(sales_test, ignore_index=True, sort=False)
# print(sales.shape)
# sales.tail().T

### Item name [category feature] (additional feature)¶

we can split it, and "one hot encode it."

In [17]:
# # Ugly code to show the idea
# from collections import Counter
# from operator import itemgetter
# # If split by '['
# items['name_1'], items['name_2'] = items['item_name'].str.split('[', 1).str
# # if split by '('
# items['name_1'], items['name_3'] = items['item_name'].str.split('(', 1).str

# items['name_2'] = items['name_2'].str.replace('[^A-Za-z0-9А-Яа-я]+', ' ').str.lower()
# items['name_3'] = items['name_3'].str.replace('[^A-Za-z0-9А-Яа-я]+', ' ').str.lower()
# items = items.fillna('0')

# # Dataframe of feature and their count if above 200
# result_1 = Counter(' '.join(items['name_2'].values.tolist()).split(' ')).items()
# # itemgetter takes keys of dictionaries and converts them into tuples
# result_1 = sorted(result_1, key=itemgetter(1))
# result_1 = pd.DataFrame(result_1, columns=['feature', 'count'])
# result_1 = result_1[(result_1['feature'].str.len() > 1) & (result_1['count'] > 200)]

# result_2 = Counter(' '.join(items['name_3'].values.tolist()).split(" ")).items()
# result_2 = sorted(result_2, key=itemgetter(1))
# result_2 = pd.DataFrame(result_2, columns=['feature', 'count'])
# result_2 = result_2[(result_2['feature'].str.len() > 1) & (result_2['count'] > 200)]

# result = pd.concat([result_1, result_2])
# result = result.drop_duplicates(subset=['feature'])

# #print('Most common aditional features:', result)

### Shops/Cats/Items preprocessing
Observations:

* Each shop_name starts with the city name.
* Each category contains type and subtype in its name.

In [18]:
shops.loc[shops.shop_name == 'Сергиев Посад ТЦ "7Я"', 'shop_name'] = 'СергиевПосад ТЦ "7Я"'
shops['city'] = shops['shop_name'].str.split(' ').map(lambda x: x[0])
shops.loc[shops.city == '!Якутск', 'city'] = 'Якутск'
shops['city_code'] = LabelEncoder().fit_transform(shops['city'])
#shops
shop_city = shops[['shop_id','city_code']].drop_duplicates()


In [19]:
category['split'] = category['item_category_name'].str.split('-')
category['type'] = category['split'].map(lambda x: x[0].strip())
category['type_code'] = LabelEncoder().fit_transform(category['type'])

category['subtype'] = category['split'].map(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
category['subtype_code'] = LabelEncoder().fit_transform(category['subtype'])
category = category[['item_category_id','type_code', 'subtype_code']]


In [20]:
# if subtype is nan then type



In [21]:
# Create "grid" with columns
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales.loc[sales['date_block_num'] == block_num, 'shop_id'].unique()
    cur_items = sales.loc[sales['date_block_num'] == block_num, 'item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

    # Turn the grid into a dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int16)

grid['date_block_num'] = grid['date_block_num'].astype(np.int8)
grid['shop_id'] = grid['shop_id'].astype(np.int8)
grid['item_id'] = grid['item_id'].astype(np.int16)


In [22]:
# Groupby data to get shop-item-month aggregates
gb = (sales.groupby(['date_block_num','shop_id','item_id'])
                 .agg({'item_cnt_day': 'sum', 'item_price':'mean' })
                 .rename(columns = {'item_cnt_day':'item_cnt_month'})
                 .reset_index())

all_data = pd.merge(grid, gb, how='left', on=index_cols).fillna(0)

all_data['item_cnt_month'] = (all_data['item_cnt_month']
                                .fillna(0)
                                .clip(0,20) # NB clip target here
                                .astype(np.float16))
#all_data = all_data.item_cnt_month.clip(0,20)
all_data.shape

(10913850, 5)

In [23]:
all_data.item_cnt_month.describe()

count    10913850.0
mean            NaN
std             0.0
min             0.0
25%             0.0
50%             0.0
75%             0.0
max            20.0
Name: item_cnt_month, dtype: float64

In [24]:
sales_test['date_block_num'] = 34
sales_test['date_block_num'] = sales_test['date_block_num'].astype(np.int8)
sales_test['shop_id'] = sales_test['shop_id'].astype(np.int8)
sales_test['item_id'] = sales_test['item_id'].astype(np.int16)

all_data = (pd.concat([all_data, sales_test], ignore_index=True, sort=False, 
                      keys=['date_block_num','shop_id','item_id']).fillna(0))
all_data.shape

(11128050, 6)

In [25]:
all_data['revenue'] = all_data['item_price'] * all_data['item_cnt_month']

In [26]:
# stop

In [27]:
all_data = pd.merge(all_data, shops, on=['shop_id'], how='left')
all_data = pd.merge(all_data, items, on=['item_id'], how='left')
all_data = pd.merge(all_data, category, on=['item_category_id'], how='left')
all_data['city_code'] = all_data['city_code'].astype(np.int8)
all_data['item_category_id'] = all_data['item_category_id'].astype(np.int8)
all_data['type_code'] = all_data['type_code'].astype(np.int8)
all_data['subtype_code'] = all_data['subtype_code'].astype(np.int8)


In [28]:
all_data.drop(['shop_name','city','item_name','ID'], axis=1, inplace=True)

In [29]:
all_data.tail(10).T


,11128040,11128041,11128042,11128043,11128044,11128045,11128046,11128047,11128048,11128049
shop_id,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0
item_id,3280.0,4393.0,4352.0,18049.0,18027.0,18454.0,16188.0,15757.0,19648.0,969.0
date_block_num,34.0,34.0,34.0,34.0,34.0,34.0,34.0,34.0,34.0,34.0
item_cnt_month,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
item_price,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
revenue,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
city_code,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0
item_category_id,55.0,22.0,22.0,70.0,70.0,55.0,64.0,55.0,40.0,37.0
type_code,13.0,5.0,5.0,14.0,14.0,13.0,14.0,13.0,11.0,11.0
subtype_code,2.0,14.0,14.0,57.0,57.0,2.0,42.0,2.0,4.0,1.0


In [30]:
all_data = lag_feature(all_data, [1,2], 'item_cnt_month')

100%|██████████| 2/2 [00:13<00:00,  6.85s/it]


### Mean encoded features

In [31]:
gb = (all_data.groupby(['date_block_num'])
                 .agg({'item_cnt_month': 'mean'})
                 .rename(columns = {'item_cnt_month':'date_avg_item_cnt'})
                 .reset_index())
all_data = pd.merge(all_data, gb, how='left', on='date_block_num')
all_data['date_avg_item_cnt'] = all_data['date_avg_item_cnt'].astype(np.float16)
all_data = lag_feature(all_data, [1], 'date_avg_item_cnt')
all_data.drop('date_avg_item_cnt', axis=1, inplace=True)


100%|██████████| 1/1 [00:06<00:00,  6.40s/it]


In [32]:
gb = (all_data.groupby(['date_block_num', 'item_id'])
                 .agg({'item_cnt_month': 'mean'})
                 .rename(columns = {'item_cnt_month':'date_item_avg_item_cnt'})
                 .reset_index())
all_data = pd.merge(all_data, gb, how='left', on=['date_block_num', 'item_id'])
all_data['date_item_avg_item_cnt'] = all_data['date_item_avg_item_cnt'].astype(np.float16)
all_data = lag_feature(all_data, [1,2], 'date_item_avg_item_cnt')
all_data.drop('date_item_avg_item_cnt', axis=1, inplace=True)


100%|██████████| 2/2 [00:13<00:00,  6.62s/it]


In [33]:
gb = (all_data.groupby(['date_block_num', 'shop_id'])
                 .agg({'item_cnt_month': 'mean'})
                 .rename(columns = {'item_cnt_month':'date_shop_avg_item_cnt'})
                 .reset_index())
all_data = pd.merge(all_data, gb, how='left', on=['date_block_num', 'shop_id'])
all_data['date_shop_avg_item_cnt'] = all_data['date_shop_avg_item_cnt'].astype(np.float16)
all_data = lag_feature(all_data, [1], 'date_shop_avg_item_cnt')
all_data.drop('date_shop_avg_item_cnt', axis=1, inplace=True)


100%|██████████| 1/1 [00:06<00:00,  6.75s/it]


In [34]:
gb = (all_data.groupby(['date_block_num', 'item_category_id'])
                 .agg({'item_cnt_month': 'mean'})
                 .rename(columns = {'item_cnt_month':'date_cat_avg_item_cnt'})
                 .reset_index())
all_data = pd.merge(all_data, gb, how='left', on=['date_block_num', 'item_category_id'])
all_data['date_cat_avg_item_cnt'] = all_data['date_cat_avg_item_cnt'].astype(np.float16)
all_data = lag_feature(all_data, [1], 'date_cat_avg_item_cnt')
all_data.drop('date_cat_avg_item_cnt', axis=1, inplace=True)


100%|██████████| 1/1 [00:07<00:00,  7.25s/it]


In [35]:
gb = (all_data.groupby(['date_block_num','shop_id','item_category_id'])
                 .agg({'item_cnt_month': 'mean'})
                 .rename(columns = {'item_cnt_month':'date_shop_cat_avg_item_cnt'})
                 .reset_index())
all_data = pd.merge(all_data, gb, how='left', on=['date_block_num','shop_id','item_category_id'])
all_data['date_shop_cat_avg_item_cnt'] = all_data['date_shop_cat_avg_item_cnt'].astype(np.float16)
all_data = lag_feature(all_data, [1], 'date_shop_cat_avg_item_cnt')
all_data.drop('date_shop_cat_avg_item_cnt', axis=1, inplace=True)


100%|██████████| 1/1 [00:10<00:00, 10.88s/it]


In [36]:
gb = (all_data.groupby(['date_block_num','shop_id','type_code'])
                 .agg({'item_cnt_month': 'mean'})
                 .rename(columns = {'item_cnt_month':'date_shop_type_avg_item_cnt'})
                 .reset_index())
all_data = pd.merge(all_data, gb, how='left', on=['date_block_num','shop_id','type_code'])
all_data['date_shop_type_avg_item_cnt'] = all_data['date_shop_type_avg_item_cnt'].astype(np.float16)
all_data = lag_feature(all_data, [1], 'date_shop_type_avg_item_cnt')
all_data.drop('date_shop_type_avg_item_cnt', axis=1, inplace=True)


100%|██████████| 1/1 [00:07<00:00,  7.85s/it]


In [37]:
gb = (all_data.groupby(['date_block_num','shop_id','subtype_code'])
                 .agg({'item_cnt_month': 'mean'})
                 .rename(columns = {'item_cnt_month':'date_shop_subtype_avg_item_cnt'})
                 .reset_index())
all_data = pd.merge(all_data, gb, how='left', on=['date_block_num','shop_id','subtype_code'])
all_data['date_shop_subtype_avg_item_cnt'] = all_data['date_shop_subtype_avg_item_cnt'].astype(np.float16)
all_data = lag_feature(all_data, [1], 'date_shop_subtype_avg_item_cnt')
all_data.drop('date_shop_subtype_avg_item_cnt', axis=1, inplace=True)


100%|██████████| 1/1 [00:07<00:00,  7.55s/it]


In [38]:
# sales.head().T
# stop
all_data.fillna(0)

,shop_id,item_id,date_block_num,item_cnt_month,item_price,revenue,city_code,item_category_id,type_code,subtype_code,item_cnt_month_lag_1,item_cnt_month_lag_2,date_avg_item_cnt_lag_1,date_item_avg_item_cnt_lag_1,date_item_avg_item_cnt_lag_2,date_shop_avg_item_cnt_lag_1,date_cat_avg_item_cnt_lag_1,date_shop_cat_avg_item_cnt_lag_1,date_shop_type_avg_item_cnt_lag_1,date_shop_subtype_avg_item_cnt_lag_1
0,59,22154,0,1.0,999.0,999.0,30,37,11,1,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,59,2552,0,0.0,0.0,0.0,30,58,13,27,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,59,2554,0,0.0,0.0,0.0,30,58,13,27,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,59,2555,0,0.0,0.0,0.0,30,56,13,3,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,59,2564,0,0.0,0.0,0.0,30,59,13,40,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11128045,45,18454,34,0.0,0.0,0.0,20,55,13,2,1.0,0.0,0.258545,0.045441,0.023254,0.126709,0.196899,0.126831,0.089294,0.126831
11128046,45,16188,34,0.0,0.0,0.0,20,64,14,42,0.0,0.0,0.258545,0.022720,0.069763,0.126709,0.155884,0.094482,0.106201,0.094482
11128047,45,15757,34,0.0,0.0,0.0,20,55,13,2,0.0,0.0,0.258545,0.113647,0.069763,0.126709,0.196899,0.126831,0.089294,0.126831
11128048,45,19648,34,0.0,0.0,0.0,20,40,11,4,0.0,0.0,0.258545,0.045441,0.069763,0.126709,0.220825,0.083740,0.097046,0.083740


In [39]:
# gb = sales.groupby(['item_id']).agg({'item_price': ['mean']})
# gb.columns = ['item_avg_item_price']
# gb.reset_index(inplace=True)

# all_data = pd.merge(all_data, gb, on=['item_id'], how='left')
# all_data['item_avg_item_price'] = all_data['item_avg_item_price'].fillna(0).astype(np.float16)

# gb = sales.groupby(['date_block_num','item_id']).agg({'item_price': ['mean']})
# gb.columns = ['date_item_avg_item_price']
# gb.reset_index(inplace=True)

# all_data = pd.merge(all_data, gb, on=['date_block_num','item_id'], how='left')
# all_data['date_item_avg_item_price'] = all_data['date_item_avg_item_price'].fillna(0).astype(np.float16)



In [40]:
# lags = [1,2,3,4,5,6]
# all_data = lag_feature(all_data, lags, 'date_item_avg_item_price')

# for i in tqdm(lags):
#     all_data['delta_price_lag_'+str(i)] = \
#         (all_data['date_item_avg_item_price_lag_'+str(i)] - 
#              all_data['item_avg_item_price']) / all_data['item_avg_item_price']


In [41]:
# def select_trend(row):
#     for i in lags:
#         if row['delta_price_lag_'+str(i)]:
#             return row['delta_price_lag_'+str(i)]
#     return 0

# all_data['delta_price_lag'] = all_data.apply(select_trend, axis=1).fillna(0).astype(np.float16)

# features_to_drop = ['item_avg_item_price', 'date_item_avg_item_price']
# for i in tqdm(lags):
#     features_to_drop += ['date_item_avg_item_price_lag_'+str(i)]
#     features_to_drop += ['delta_price_lag_'+str(i)]

# all_data.drop(features_to_drop, axis=1, inplace=True)


In [42]:
gb = sales.groupby(['date_block_num','shop_id']).agg({'revenue': ['sum']})
gb.columns = ['date_shop_revenue']
gb.reset_index(inplace=True)

all_data = pd.merge(all_data, gb, on=['date_block_num','shop_id'], how='left')
all_data['date_shop_revenue'] = all_data['date_shop_revenue'].astype(np.float32)

gb = gb.groupby(['shop_id']).agg({'date_shop_revenue': ['mean']})
gb.columns = ['shop_avg_revenue']
gb.reset_index(inplace=True)

all_data = pd.merge(all_data, gb, on=['shop_id'], how='left')
all_data['shop_avg_revenue'] = all_data['shop_avg_revenue'].astype(np.float32)

all_data['delta_revenue'] = (all_data['date_shop_revenue'] - all_data['shop_avg_revenue']) / all_data['shop_avg_revenue']
all_data['delta_revenue'] = all_data['delta_revenue'].astype(np.float16)

all_data = lag_feature(all_data, [1], 'delta_revenue')

all_data.drop(['date_shop_revenue','shop_avg_revenue','delta_revenue'], axis=1, inplace=True)


100%|██████████| 1/1 [00:10<00:00, 10.13s/it]


In [43]:
all_data['month'] = all_data['date_block_num'] % 12

days = pd.Series([31,28,31,30,31,30,31,31,30,31,30,31])
all_data['days'] = all_data['month'].map(days).astype(np.int8)

In [44]:
cache = {}
all_data['item_shop_last_sale'] = -1
all_data['item_shop_last_sale'] = all_data['item_shop_last_sale'].astype(np.int8)
for idx, row in tqdm(all_data.iterrows()):    
    key = str(row.item_id)+' '+str(row.shop_id)
    if key not in cache:
        if row.item_cnt_month!=0:
            cache[key] = row.date_block_num
    else:
        last_date_block_num = cache[key]
        all_data.at[idx, 'item_shop_last_sale'] = row.date_block_num - last_date_block_num
        cache[key] = row.date_block_num         


11128050it [15:11, 12210.93it/s]


In [45]:
cache = {}
all_data['item_last_sale'] = -1
all_data['item_last_sale'] = all_data['item_last_sale'].astype(np.int8)
for idx, row in tqdm(all_data.iterrows()):    
    key = row.item_id
    if key not in cache:
        if row.item_cnt_month!=0:
            cache[key] = row.date_block_num
    else:
        last_date_block_num = cache[key]
        if row.date_block_num>last_date_block_num:
            all_data.at[idx, 'item_last_sale'] = row.date_block_num - last_date_block_num
            cache[key] = row.date_block_num         


11128050it [09:39, 19216.57it/s]


In [46]:
all_data['item_shop_first_sale'] = all_data['date_block_num'] - all_data.groupby(['item_id','shop_id'])['date_block_num'].transform('min')
all_data['item_first_sale'] = all_data['date_block_num'] - all_data.groupby('item_id')['date_block_num'].transform('min')


In [47]:
all_data = all_data[all_data.date_block_num > 11]
all_data.shape

(6639294, 27)

In [48]:
all_data.head(2).T

,4488756,4488757
shop_id,54.000000,54.000000
item_id,10297.000000,10296.000000
date_block_num,12.000000,12.000000
item_cnt_month,4.000000,3.000000
item_price,749.000000,1599.000000
revenue,2996.000000,4796.000000
city_code,26.000000,26.000000
item_category_id,37.000000,38.000000
type_code,11.000000,11.000000
subtype_code,1.000000,1.000000


In [49]:
# stop

In [50]:
all_data = all_data.rename(columns = {'item_cnt_month':'target'})
all_data.tail(2).T

,11128048,11128049
shop_id,45.000000,45.000000
item_id,19648.000000,969.000000
date_block_num,34.000000,34.000000
target,0.000000,0.000000
item_price,0.000000,0.000000
revenue,0.000000,0.000000
city_code,20.000000,20.000000
item_category_id,40.000000,37.000000
type_code,11.000000,11.000000
subtype_code,4.000000,1.000000


In [51]:
all_data = downcast_dtypes(all_data)
all_data['shop_id'] = all_data['shop_id'].astype(np.int8)
all_data['date_block_num'] = all_data['date_block_num'].astype(np.int8)
all_data['target'] = all_data['target'].astype(np.int8)
all_data['type_code'] = all_data['type_code'].astype(np.int8)
all_data['subtype_code'] = all_data['subtype_code'].astype(np.int8)
all_data['city_code'] = all_data['city_code'].astype(np.int8)


In [52]:
all_data.to_pickle('all_data.pkl')



In [53]:
del grid,gb,sales,sales_test,train,shops,items,category,cache
gc.collect()

62

In [54]:
# Restart afterr kernal crash
import pandas as pd
import numpy as np
from xgboost import XGBRegressor

import gc
import pickle


all_data = pd.read_pickle('all_data.pkl')
test = pd.read_csv('./final_project_data/test.csv')
sales_index = test['ID']
del test
gc.collect()

20

In [55]:
all_data.head(2).T

,4488756,4488757
shop_id,54.000000,54.000000
item_id,10297.000000,10296.000000
date_block_num,12.000000,12.000000
target,4.000000,3.000000
item_price,749.000000,1599.000000
revenue,2996.000000,4796.000000
city_code,26.000000,26.000000
item_category_id,37.000000,38.000000
type_code,11.000000,11.000000
subtype_code,1.000000,1.000000


### train/ validation/test split

34th month data is the test set. 32nd and 33rd data will be taken as validation split and rest as training data.

In [56]:
# Save `date_block_num`, as we can't use them as features, but will need them to split the dataset into parts 
dates = all_data['date_block_num']
to_drop_cols = ['target', 'item_price']

X_train = all_data.loc[(dates <  32)].drop(to_drop_cols, axis=1)
X_val = all_data.loc[(dates ==  33) | (dates ==  32)].drop(to_drop_cols, axis=1)
X_test =  all_data.loc[dates == 34].drop(to_drop_cols, axis=1)

y_train = all_data.loc[(dates <  32), 'target'].values
y_val =  all_data.loc[((dates ==  33) | (dates ==  32)), 'target'].values

gc.collect();

In [57]:
# from keras import backend as K

# def root_mean_squared_error(y_true, y_pred):
#         return K.sqrt(K.mean(K.square(y_pred - y_true))) 


In [58]:
# def build_model():
#     model = keras.Sequential([
#         layers.Dense(128, activation='relu',input_shape=[X_train.shape[1]]),
#         layers.Dense(128, activation='relu'),
#         layers.Dense(64, activation='relu'),
#         layers.Dense(32, activation='relu'),
#        layers.Dense(1)
#       ])

# #     optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.0)
#     optimizer='rmsprop'
# #     optimizer='adam'
    
#     model.compile(loss=root_mean_squared_error,
#                 optimizer=optimizer,
#                 metrics=['mse',])
#     return model

In [59]:
# keras_model = build_model()

In [60]:
# EPOCHS = 10

# history = keras_model.fit(
#   X_train, y_train,
#   epochs=EPOCHS,
#     batch_size=100,
#     validation_data=(X_val, y_val),
#     #callbacks=[callbacks],
#   )

In [61]:
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('Model accuracy')
# plt.ylabel('loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'val'], loc='upper left')
# # plt.ylim(bottom=10, top=12)
# # plt.xlim(left=1100, right = 1200)
# plt.show()

In [62]:
# # Preprocessing of test data, fit model
# preds_test = keras_model.predict(X_test)

# # The prediction is of type numpy.ndarray
# preds_list = preds_test.tolist()

# # Extract the prediction and put it in a list
# prediction = []
# for item in preds_list:
#     prediction.append(item[0])

# prediction = np.clip(prediction, 0, 20)


In [63]:
xgb_model = XGBRegressor(
    max_depth=8,
    n_estimators=1000,
    min_child_weight=300, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    eta=0.3,    
    seed=42)


In [ ]:
xgb_model.fit(
    X_train, 
    y_train, 
    eval_metric="rmse", 
    eval_set=[(X_train, y_train), (X_val, y_val)], 
    verbose=True, 
    early_stopping_rounds = 3)


In [ ]:
preds = xgb_model.predict(X_test).clip(0, 20)

prediction = (np.clip(preds, 0, 20)).tolist()

In [ ]:
# Save test predictions to file
output = pd.DataFrame({'ID': sales_index,
                       'item_cnt_month': prediction})
# output.loc[output.item_cnt_month < 0.05, 'item_cnt_month'] = 0

In [ ]:
# output.to_csv('XbgModel.csv', index=False)
# output.to_csv('KerasModel.csv', index=False)
output.to_csv('XGBModel.csv', index=False)

In [ ]:
print(output.item_cnt_month.describe())
print(output.item_cnt_month.sum())